In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, KFold
from lightgbm import LGBMClassifier

import gc

## Import paths

In [2]:
root_dir = '/home/vivek/Datasets/'
project_dir = 'Building_Damage_Classification/'
file_path = os.path.join(root_dir,project_dir)

In [3]:
buil_owner_path = os.path.join(file_path, "Building_Ownership_Use.csv")
buil_stru_path = os.path.join(file_path, "Building_Structure.csv")
train_path = os.path.join(file_path, "train.csv")
test_path = os.path.join(file_path, "test.csv")

In [4]:
buil_owner = pd.read_csv(buil_owner_path)
buil_stru = pd.read_csv(buil_stru_path)
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [5]:
buil_stru.drop(buil_stru.columns[0], axis=1, inplace=True)

In [6]:
combined = pd.concat([buil_owner, buil_stru], axis=1)
combined_new = combined

In [7]:
del buil_stru, buil_owner
gc.collect()

18

In [8]:
combined_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052948 entries, 0 to 1052947
Data columns (total 45 columns):
building_id                               1052948 non-null object
district_id                               1052948 non-null int64
vdcmun_id                                 1052948 non-null int64
ward_id                                   1052948 non-null int64
legal_ownership_status                    1052948 non-null object
count_families                            1052946 non-null float64
has_secondary_use                         1052938 non-null float64
has_secondary_use_agriculture             1052948 non-null int64
has_secondary_use_hotel                   1052948 non-null int64
has_secondary_use_rental                  1052948 non-null int64
has_secondary_use_institution             1052948 non-null int64
has_secondary_use_school                  1052948 non-null int64
has_secondary_use_industry                1052948 non-null int64
has_secondary_use_health_post       

Use dummies to convert all the `object` features.

In [9]:
# For "land_surface_condition" take a look at how many classes we have
combined_new.land_surface_condition.value_counts()

Flat              874202
Moderate slope    144748
Steep slope        33998
Name: land_surface_condition, dtype: int64

In [10]:
combined_new.foundation_type.value_counts()

Mud mortar-Stone/Brick    886331
Bamboo/Timber              60598
Cement-Stone/Brick         57354
RC                         42808
Other                       5857
Name: foundation_type, dtype: int64

In [11]:
combined_new.roof_type.value_counts()

Bamboo/Timber-Light roof    739184
Bamboo/Timber-Heavy roof    248235
RCC/RB/RBC                   65529
Name: roof_type, dtype: int64

In [12]:
combined_new.ground_floor_type.value_counts()

Mud            847250
Brick/Stone    100487
RC              99046
Timber           4072
Other            2093
Name: ground_floor_type, dtype: int64

In [13]:
combined_new.other_floor_type.value_counts()

TImber/Bamboo-Mud    666182
Timber-Planck        177571
Not applicable       160301
RCC/RB/RBC            48894
Name: other_floor_type, dtype: int64

In [14]:
combined_new.position.value_counts()

Not attached       816364
Attached-1 side    173459
Attached-2 side     53522
Attached-3 side      9602
Name: position, dtype: int64

In [15]:
combined_new.plan_configuration.value_counts()

Rectangular                        1009987
Square                               23106
L-shape                              15088
Multi-projected                       1412
T-shape                               1302
Others                                 994
U-shape                                591
Building with Central Courtyard        203
E-shape                                167
H-shape                                 97
Name: plan_configuration, dtype: int64

In [16]:
combined_new.legal_ownership_status.value_counts()

Private          1014042
Public             22326
Institutional      10502
Other               6078
Name: legal_ownership_status, dtype: int64

In [17]:
combined_new.condition_post_eq.value_counts()

Damaged-Not used                           249464
Damaged-Repaired and used                  211119
Damaged-Used in risk                       187480
Damaged-Rubble unclear                     145732
Damaged-Rubble clear                       132508
Not damaged                                 71587
Damaged-Rubble Clear-New building built     54587
Covered by landslide                          471
Name: condition_post_eq, dtype: int64

In [18]:
land_dum = pd.get_dummies(combined_new.land_surface_condition)
foun = pd.get_dummies(combined_new.foundation_type)
roof = pd.get_dummies(combined_new.roof_type)
grnd = pd.get_dummies(combined_new.ground_floor_type)
oth = pd.get_dummies(combined_new.other_floor_type)
pos = pd.get_dummies(combined_new.position)
plan = pd.get_dummies(combined_new.plan_configuration)
con = pd.get_dummies(combined_new.condition_post_eq)
legal = pd.get_dummies(combined_new.legal_ownership_status)

In [19]:
cols = [land_dum, foun, roof, grnd, oth, pos, plan, con, legal]

In [20]:
dummy_cols = pd.concat(cols, axis=1)

In [21]:
del land_dum, foun, roof, grnd, oth, pos, plan, con, legal
gc.collect()

21

In [22]:
building_id = combined_new.building_id

In [23]:
drop_cols = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration',
            'condition_post_eq', 'legal_ownership_status','district_id', 'vdcmun_id','building_id','ward_id']

In [24]:
combined_new.drop(drop_cols, axis=1, inplace=True)

### Combining main data frame with dummy columns

In [25]:
combined_new = pd.concat([combined_new, dummy_cols], axis=1)

In [26]:
combined_new = combined_new.astype(float)
combined_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052948 entries, 0 to 1052947
Data columns (total 75 columns):
count_families                             1052946 non-null float64
has_secondary_use                          1052938 non-null float64
has_secondary_use_agriculture              1052948 non-null float64
has_secondary_use_hotel                    1052948 non-null float64
has_secondary_use_rental                   1052948 non-null float64
has_secondary_use_institution              1052948 non-null float64
has_secondary_use_school                   1052948 non-null float64
has_secondary_use_industry                 1052948 non-null float64
has_secondary_use_health_post              1052948 non-null float64
has_secondary_use_gov_office               1052948 non-null float64
has_secondary_use_use_police               1052948 non-null float64
has_secondary_use_other                    1052948 non-null float64
count_floors_pre_eq                        1052948 non-null float64
cou

In [27]:
# add building id to our dataframe
combined_new = pd.concat([building_id, combined_new], axis=1)
combined_new.set_index('building_id',inplace=True)

In [28]:
combined_new.head()

,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,...,Damaged-Repaired and used,Damaged-Rubble Clear-New building built,Damaged-Rubble clear,Damaged-Rubble unclear,Damaged-Used in risk,Not damaged,Institutional,Other,Private,Public
building_id,,,,,,,,,,,,,,,,,,,,,
a3380c4f75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a3380c4fd9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a3380c503e,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a338a4e5f2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
a338a4e653,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Now combine this to train and test files

train.info()

In [29]:
test.head(5)

,area_assesed,building_id,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,a3380c4f75,7,0.0,0,0,0,0,0,0,0,1.0,701
1,Both,a338a4e653,7,0.0,0,0,0,0,0,0,0,1.0,701
2,Building removed,a338a4e6b7,7,0.0,0,0,0,0,0,0,0,1.0,701
3,Both,a33a6eaa3a,7,0.0,0,0,0,0,0,0,0,1.0,701
4,Building removed,a33b073ff6,7,0.0,0,0,0,0,0,0,0,1.0,701


In [30]:
test_df = pd.DataFrame([])
for i in range(len(test)):
    rows = combined_new.loc[[test.building_id[i]]]
    test_df = test_df.append(rows)
    if i%10000==0:
        print(int(i*100/len(train)), '% records')

0 % records
1 % records
3 % records


KeyboardInterrupt: 

In [ ]:
test['damage_grade'] = -111
trn_tst = pd.concat((train, test), axis=0)
trn_tst.shape

In [ ]:
trn_tst.info()

In [ ]:
trn_tst['area_assesed'].value_counts()

In [ ]:
trn_tst = pd.concat([trn_tst, pd.get_dummies(trn_tst.area_assesed)], axis=1).drop('area_assesed', axis=1)

In [ ]:
full = pd.merge(stru_owner, trn_tst,on=['building_id','district_id','vdcmun_id'])
full.info()

We have only one feature variable which contains missing values.
Let's check out this values and we will use `Imputer` to fill the missing values.

In [ ]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values="NaN", strategy='most_frequent')
imputed_values = full.has_repair_started.values.reshape(-1,1)
full.has_repair_started = imp.fit_transform(imputed_values, full.has_repair_started)

Let's check is there any null values in our full dataframe.

In [ ]:
full.isnull().sum()

Check the most frequent values in `count_families` variable.

In [ ]:
full.count_families.value_counts()

In [ ]:
imp1 = Imputer(missing_values="NaN", strategy="most_frequent")
imputed_values1 = full.count_families.values.reshape(-1,1)
full.count_families = imp1.fit_transform(imputed_values1, full.count_families)

In [ ]:
# let's check is there any null values in our dataframe
full.isnull().sum().sum()

In [ ]:
full.shape

### Dropping and transforming column values

Set the `building_id` as index

In [ ]:
full = full.set_index('building_id')
full.head(2)

Dropping the columns which are not required.

In [ ]:
full = full.drop(['district_id','vdcmun_id', 'ward_id'], axis=1)
full.info()

### Reorder columns and Normalize of Data

In [ ]:
column = [column for column in full.columns if column != 'damage_grade']
columns = ['damage_grade'] + column

In [ ]:
full = full[columns]
full.info()

In [ ]:
full.head(2)

In [ ]:
del trn_tst, stru_owner, buil_owner, buil_stru, train, test
gc.collect()

## Saving processed data

In [ ]:
processed_data_path = '/home/vivek/Git/building_damage/'
train_path = os.path.join(processed_data_path, 'train.csv')
test_path = os.path.join(processed_data_path, 'test.csv')

In [ ]:
# train data
full.loc[full['damage_grade'] != -111].to_csv(train_path)

# test data
full.loc[full['damage_grade'] == -111].to_csv(test_path)